In [1]:
import geopandas as gpd
import xarray as xr
import pandas as pd

import pint_xarray
import pint_pandas
import pint

from summaflow import (
    GeoLayer,
    SUMMAWorkflow,
)

import os
import glob

In [2]:
# paths
root_path = '/Users/kasrakeshavarz/Documents/github-repos/bow-river-at-banff/bb-model/'

# layers' paths
landcover_path = os.path.join(root_path, 'attributes', 'landcover', 'MCD12Q1.061')
soilclass_path = os.path.join(root_path, 'attributes', 'soil')
merithdyr_path = os.path.join(root_path, 'attributes', 'elevation')

# geolayer's path
riv_path = os.path.join(root_path, 'shapefiles', 'bb_rivers.shp')
cat_path = os.path.join(root_path, 'shapefiles', 'bb_subbasins.shp')
hru_path = os.path.join(root_path, 'shapefiles', 'bb_subbasins.shp')

# forcings path
root_path_forcings = os.path.join(root_path, 'forcing', 'remapped')

In [3]:
# Geospatial layers
riv_obj = gpd.read_file(riv_path)
cat_obj = gpd.read_file(cat_path)
hru_obj = gpd.read_file(hru_path)

In [4]:
# layers needed by the setup workflow
# elevation
elv = GeoLayer.from_maf(
    maf_stats=os.path.join(merithdyr_path, 'bb_model_stats_elv.csv'),
    maf_layer=os.path.join(merithdyr_path, 'bb_model_elv.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'meters',
)
# landcover
landcover = GeoLayer.from_maf(
    maf_stats=os.path.join(landcover_path, 'bb_model_stats_MCD12Q1.061_2022.csv'),
    maf_layer=os.path.join(landcover_path, 'bb_model_2022.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'dimensionless',
)
# USDA soil classes
soil = GeoLayer.from_maf(
    maf_stats=os.path.join(soilclass_path, 'bb_model_stats_soil_classes.csv'),
    maf_layer=os.path.join(soilclass_path, 'bb_model_soil_classes.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'dimensionless',
)

# dummy layers for `tan_slope`, `contourLength` and `downHRUindex`
# until relevant workflows are implemented inside `gistool`
slope = elv
contour = elv
hru_index = elv

In [5]:
exp = SUMMAWorkflow(
    forcing_data = glob.glob(os.path.join(root_path_forcings, '**', '*.nc'), recursive=True),
    forcing_name_mapping = {
        'RDRS_v2.1_A_PR0_SFC': 'pptrate',
        'RDRS_v2.1_P_TT_09944': 'airtemp',
        'RDRS_v2.1_P_P0_SFC': 'airpres',
        'RDRS_v2.1_P_FI_SFC': 'LWRadAtm',
        'RDRS_v2.1_P_FB_SFC': 'SWRadAtm',
        'RDRS_v2.1_P_HU_09944': 'spechum',
        'RDRS_v2.1_P_UVC_09944': 'windspd',
    },
    forcing_unit_mapping = {
        'pptrate': 'meter / hour',
        'airtemp': 'degC',
        'airpres': 'millibar',
        'LWRadAtm': 'watt / meter ** 2',
        'SWRadAtm': 'watt / meter ** 2',
        'spechum': 'dimensionless',
        'windspd': 'knot',
    },
    forcing_to_unit_mapping = {
        'pptrate': 'millimeter / second',
        'airtemp': 'kelvin',
        'airpres': 'pascal',
        'LWRadAtm': 'watt / meter ** 2',
        'SWRadAtm': 'watt / meter ** 2',
        'spechum': 'dimensionless',
        'windspd': 'meter / second',
    },
    forcing_attrs = {
        'measurement_height': 40,
        'measurement_height_unit': 'meters',
        'forcing_time_zone': 'utc', # original timezone of the forcing datatset
        'target_time_zone': 'utc', # if UTC, SUMMA converts to local time zone internally
        'local': {
            'pptrate': {
                'units': 'mm/s'
            },
        },
        'global': {
            'Created for': 'Bow River Basin',
        },
    },
    topology_data = {
        'riv': riv_obj,
        'hru': hru_obj,
        'cat': cat_obj,
    },
    topology_unit_mapping = {'a': 'm', 'c': 'ha'},
    topology_to_unit_mapping = {'a': 'bar', 'c': 'dca'},
    topology_attrs = {
        'gru_fid': 'COMID',
        'hru_fid': 'COMID',
        'local': {},
        'global': {},
    },
    geospatial_data = {
        'elevation': elv,
        'soilTypeIndex': soil,
        'vegTypeIndex': landcover,
        'tan_slope': slope,
        'contourLength': contour,
        'downHRUindex': hru_index,
    },
    cold_state = {
        'layers': {
            'nSoil': 8,
            'nSnow': 0,
        },
        'states': {
            'scalarCanopyIce': 0,
            'scalarCanopyLiq': 0,
            'scalarSnowDepth': 0,
            'scalarSWE': 0,
            'scalarSfcMeltPond': 0,
            'scalarAquiferStorage': 0.4,
            'scalarSnowAlbedo': 0,
            'scalarCanairTemp': 283.16,
            'scalarCanopyTemp': 283.16,
            'mLayerTemp': 283.16,
            'mLayerVolFracIce': 0,
            'mLayerVolFracLiq': 0.4,
            'mLayerMatricHead': -1.0,
            'mLayerDepth': [0.025, 0.075, 0.15, 0.25, 0.5, 0.5, 1, 1.5],
        },
    },
    decisions = { # Can change all decisions, otherwise default values
        'soilCatTbl': 'ROSETTA',
    },
    auxillary = {
        # 'dt_init': 450 # if not provided, defaults to forcing data timestep
    },
    settings = {
        'model_path': '/Users/kasrakeshavarz/Documents/github-repos/bow-river-at-banff/bb-model/',
        'start_date': '1980-01-01 13:00',
        'end_date': '1980-01-05 10:00',
        'verbose': True,
    },
    fillna = {
        'geospatial_data': {
            'elevation': 1, # a rough assumption--can be modified to anything
            'soilTypeIndex': 6, # based on Darri's assumption--can be modified to anything
            'vegTypeIndex': 1, # a rough assumption--can be modified to anything
        },
    },
)

2025-05-30 13:31:12,388 - summaflow.core - INFO - SUMMA workflow initialized


/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/src/summaflow/core.py:266: UserWarning: `dt_init` not provided in auxillary dictionary; defaulting to forcing time-step.
  warnings.warn("`dt_init` not provided in auxillary dictionary;"


## Workflow tests

In [6]:
exp.run(save=True)

2025-05-30 13:31:12,390 - summaflow.core - INFO - Running SUMMA workflow
2025-05-30 13:31:12,409 - summaflow.core - INFO - Initializing attributes for SUMMA workflow...
2025-05-30 13:31:12,410 - summaflow.core - INFO - Creating attributes xarray.Dataset
2025-05-30 13:31:12,410 - summaflow.core - INFO - Adding `mHeight` attribute
2025-05-30 13:31:12,411 - summaflow.core - INFO - Adding `slopeTypeIndex` attribute
2025-05-30 13:31:12,411 - summaflow.core - INFO - Adding `hruId` and `gruId` attributes
2025-05-30 13:31:12,412 - summaflow.core - INFO - Adding `hru2gruId` attributes
2025-05-30 13:31:12,414 - summaflow.core - INFO - Calculating and adding `latitude` and `logitude` attributes
2025-05-30 13:31:12,421 - summaflow.core - INFO - Calculating and adding `area` attributes
2025-05-30 13:31:12,513 - summaflow.core - INFO - Adding geospatial layers' attributes
2025-05-30 13:31:12,513 - summaflow.core - INFO - Adding `tan_slope` attributes
2025-05-30 13:31:12,514 - summaflow.core - INFO -

/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/src/summaflow/utils.py:138: UserWarning: Using EPSG:3857 for centroid calculations on geographic data. For better accuracy, specify a local projection using calculation_crs parameter.
  warnings.warn(


2025-05-30 13:31:12,641 - summaflow.core - INFO - Initializing attributes for SUMMA workflow...
2025-05-30 13:31:12,642 - summaflow.core - INFO - Assigning timezone
2025-05-30 13:31:12,642 - summaflow.core - INFO - Processing forcing file: remapped_remapped_bb_model_1980010112.nc
2025-05-30 13:31:12,654 - summaflow.core - INFO - Saving dataset to /Users/kasrakeshavarz/Documents/github-repos/bow-river-at-banff/bb-model/forcing/SUMMA/remapped_remapped_bb_model_1980010112.nc
2025-05-30 13:31:12,662 - summaflow.core - INFO - Processing forcing file: remapped_remapped_bb_model_1980010212.nc
2025-05-30 13:31:12,668 - summaflow.core - INFO - Saving dataset to /Users/kasrakeshavarz/Documents/github-repos/bow-river-at-banff/bb-model/forcing/SUMMA/remapped_remapped_bb_model_1980010212.nc
2025-05-30 13:31:12,676 - summaflow.core - INFO - Processing forcing file: remapped_remapped_bb_model_1980010312.nc
2025-05-30 13:31:12,682 - summaflow.core - INFO - Saving dataset to /Users/kasrakeshavarz/Docum